# Cogito
αβκτν

## Introduction
### Definitions
- Let `X` be a `m`-by-`n` matrix of input data, and `y` be a `m`-by-1 vector of output data, where `m` is the # of data points, and `n` is the # of features, so that `X` represents the features and `y` represents the labels.
- Let `w` be a `n`-by-1 vector of weights, and `b` be a scalar of bias.
- Let `dw` be the loss gradient w.r.t. `w`, and `db` be the loss gradient w.r.t. `b`, where for least squares we have ${dw=\frac{1}{m}\sum_{i=1}^{m}(ŷ^{(i)} - y^{(i)})x^{(i)}}$ and ${db = \frac{1}{m}\sum_{i=1}^{m}(ŷ^{(i)} - y^{(i)})}$.
- Let `β` be the batch size.
- Let `τ` be the # of epochs.
- Let `α` be the learning rate.
- Let `κ` be the # of components (where `n` is the maximum possible # of components).
- Let `ν` be the variance ratio.

### Naming Conventions
1. Dataset is treated as a single word.
2. Featureset refers to the features of a dataset.
3. Labelset refers to the labels of a dataset.

## Setup
### Packages
1. numpy
2. pandas
3. sklearn (scikit-learn)

In [1]:
# Import os for operating system functions.
import os
# Import numpy for mathematical computation.
import numpy as np
# Import pandas for data manipulation.
import pandas as pd
# Import datetime for local time retrieval.
from datetime import datetime
# Import logging for file logging.
import logging
# Import time for duration measurement.
import time
# Import PCA and StandardScaler for PCA.
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# Import KNeighborsClassifier for KNN training.
from sklearn.neighbors import KNeighborsClassifier

# Initialise file logging configuration.
logging.basicConfig(filename="main.log", level=logging.INFO, format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")

# Load train and test datasets.
S_train = pd.read_csv("train.csv")
S_train_tfidf = pd.read_csv("train_tfidf_features.csv")
S_test = pd.read_csv("test.csv")
S_test_tfidf = pd.read_csv("test_tfidf_features.csv")

# Extract train features, train labels, and test features.
X_train = S_train_tfidf.iloc[:, 2:].values
y_train = S_train["label"].values.reshape(-1, 1)
X_test = S_test_tfidf.iloc[:, 1:].values

## Task 1: Logistic Regression Model
- Mini-batch gradient descent is used for training.

In [ ]:
# Return σ(z) for some z.
def σ(z): return 1 / (1 + np.exp(-z))

# Return cross-entropy loss for some y and ŷ.
def loss(y, ŷ):
	m, _ = y.shape
	return -1/m * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, and ŷ.
def gradients(X, y, ŷ):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	return dw, db

# Return w and b from batched gradient descent on X and y, for batch size β, epochs τ, and learning rate α.
def train_logreg(X, y, β, τ, α):
	m, n = X.shape
	# Initialise w to 0 vector and b to 0.
	w, b = np.zeros((n, 1)), 0
	# For each epoch:
	for epoch in range(τ):
		# For every batch within epoch:
		for i in range(0, m, β):
			X_batch, y_batch = X[i:i+β], y[i:i+β]
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients(X_batch, y_batch, ŷ)
			w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(X, w, b):
	# Compute ŷ = σ(w⋅X + b).
	ŷ = σ(np.dot(X, w) + b)
	return [1 if p > 0.5 else 0 for p in ŷ]

# Train model, make model predictions, and save model predictions to CSV file, for some β, τ, and α.
def generate_predictions_logreg(X_train, y_train, X_test, β, τ, α):
	start_time = time.time()
	w, b = train_logreg(X_train, y_train, β, τ, α)
	os.makedirs("./predictions/logreg/", exist_ok=True)
	file_name = os.path.join("./predictions/logreg/", f"predictions_logreg(β={β}_τ={τ}_α={α}).csv")
	pd.DataFrame({
		"id": S_test["id"],
		"label": predict_logreg(X_test, w, b)
	}).to_csv(file_name, index=False)
	end_time = time.time()
	# Log and print success message.
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

generate_predictions_logreg(X_train, y_train, X_test, 128, 10000, 0.05)

## Task 2: PCA-KNN Model
- PCA is used to reduce on features.
- KNN is used for training.

Usage Examples:
1. `generate_predictions_pcaknn(X_train, y_train, X_test, 0.95, "ν")`
2. `generate_predictions_pcaknn(X_train, y_train, X_test, 2000, "κ")`

In [ ]:
# Return PCA-transformed train and test featuresets, for some ν or κ.
def apply_pca(X_train, X_test, mode_value, mode):
	# Validate mode and mode value.
	if mode not in ["ν", "κ"] or (mode == "ν" and not (0 < mode_value <= 1)) or (mode == "κ" and not (mode_value > 1 and isinstance(mode_value, int))): raise ValueError("Cogito: mode must either be ν with value between 0 and 1, or κ with mode_value as an integer greater than 1.")
	# Instantiate PCA and StandardScaler objects.
	pca = PCA(n_components=mode_value)
	scaler = StandardScaler()
	# PCA-transform train and test featuresets.
	X_train_pca = pca.fit_transform(scaler.fit_transform(X_train))
	X_test_pca = pca.transform(scaler.transform(X_test))
	# Return PCA-transformed featuresets and # of components.
	return X_train_pca, X_test_pca, pca.n_components_

# Return KNN predictions for PCA-transformed train and test datasets.
def train_and_predict_knn(X_train_pca, y_train, X_test_pca):
	# Instantiate KNeighborsClassifier object.
	knn = KNeighborsClassifier(n_neighbors=2)
	knn.fit(X_train_pca, y_train)
	return knn.predict(X_test_pca)

# Train model, make model predictions, and save model predictions to CSV file, for some ν or κ.
def generate_predictions_pcaknn(X_train, y_train, X_test, mode_value, mode):
	# Validate mode and mode value.
	if mode not in ["ν", "κ"] or (mode == "ν" and not (0 < mode_value <= 1)) or (mode == "κ" and not (mode_value > 1 and isinstance(mode_value, int))): raise ValueError("Cogito: mode must either be ν with value between 0 and 1, or κ with mode_value as an integer greater than 1.")

	start_time = time.time()
	X_train_pca, X_test_pca, n_components = apply_pca(X_train, X_test, mode_value, mode)
	os.makedirs("./predictions/pcaknn/", exist_ok=True)
	if mode == "ν": file_name = os.path.join("./predictions/pcaknn/", f"predictions_pcaknn_ν={mode_value}_κ={n_components}.csv")
	else: file_name = os.path.join("./predictions/pcaknn/", f"predictions_pcaknn(κ={n_components}).csv")
	pd.DataFrame({
		"id": S_test["id"],
		"label": train_and_predict_knn(X_train_pca, y_train, X_test_pca)
	}).to_csv(file_name, index=False)
	end_time = time.time()
	# Log and print success message.
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

# generate_predictions_pcaknn(X_train, y_train, X_test, 5000, "κ")